In [2]:
import ginza # pip install -U ginza ja_ginza
import nltk  # pip install nltk
import spacy # pip install nltk
import pke   # pip install git+https://github.com/boudinfl/pke.git
import pandas as pd

In [3]:
df = pd.read_csv('datasets/bachelor_themes.csv')
df.head()

,年度,学科／研究科,研究室,学籍番号,賞,氏名,要旨,題目
0,H17,情報科学科,相研,NaN,NaN,NaN,NaN,WEBページにおけるユーザビリティの追及
1,H17,情報科学科,松永研,NaN,NaN,NaN,NaN,安全運転管理教育システム(ASSIST)における複数カメラ使用での交通事故防止対策
2,H17,情報科学科,宮崎研,NaN,NaN,NaN,NaN,電子透かしの体制に関する検討 -加重平均フィルタ・メディアンフィルタの場合-
3,H17,情報科学科,成研,NaN,NaN,NaN,NaN,対面教育を支援するためのウェブ助言システム
4,H17,情報科学科,石田健研,NaN,NaN,NaN,NaN,衛星通信検討のための気象データの解析-2002年のデータ解析-


In [4]:
target_labs = [
    '朝廣研','安部研','石田健研','合志研','下川研',
    '成研','田中研','仲研','米元研','アプドゥハン研',
    '稲永研','于研','石田俊研','澤田研','安武研','前田研']

target_df = df[df["研究室"].isin(target_labs )] 
target_df.head() 

,年度,学科／研究科,研究室,学籍番号,賞,氏名,要旨,題目
3,H17,情報科学科,成研,NaN,NaN,NaN,NaN,対面教育を支援するためのウェブ助言システム
4,H17,情報科学科,石田健研,NaN,NaN,NaN,NaN,衛星通信検討のための気象データの解析-2002年のデータ解析-
6,H17,情報科学科,仲研,NaN,NaN,NaN,NaN,生化学反応シミュレータの比較評価
7,H17,情報科学科,澤田研,NaN,NaN,NaN,NaN,FPGAを搭載したロボット教材のためのIPとしてKUE-CHIP2の検討
9,H17,情報科学科,石田健研,NaN,NaN,NaN,NaN,重畳積文法によるX線CTの画像再構成-Shepp-Loganのフィルタ関数の適用-


In [5]:
years = df["年度"].unique()
years

array(['H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'H24', 'H25',
       'H26', 'H27', 'H28', 'H29', 'H30', 'R01', 'R02', 'R03', 'R04'],
      dtype=object)

In [6]:
docs = {}
for y in years:
    for b in target_labs:
        titles = target_df[(target_df['年度']==y) & (target_df['研究室']==b)]['題目'].values.tolist()
        # print(f"{y}:{b}")
        docs[(y, b)] = titles

r04_ishida = docs['R04', '石田健研']
print('。'.join(r04_ishida))

MathematicaとMathTeXを用いた線形代数の教材作成 ―4次の行列式、クラーメルの公式―。MathematicaとMathTeXを用いた微分積分の教材作成 ―合成関数の微分と有理数べき関数の追加―。プログラミング環境による連立一次方程式求解の速度比較。最小点の探索におけるアルゴリズムの調査 ―探索アルゴリズムによる実行時間の比較―。層状媒質の1次元FDTD法解析 ―パルスの反射波から層状媒質の比誘電率分布の推定―。マイクロ波を用いる誘電体円柱の誘電率推定に対するMATLABニューラルネットワークの検証。九産大12号館建物内に侵入したローカル5Gの電波測定


In [7]:
pke.lang.stopwords['ja_ginza'] = 'ja'
def get_key_phrase(spacy_model, text, n = 5, stopwords=None):
    """キーフレーズ処理

    Args:
        text ([type]): キーフレーズ対象の文字列
        n (int, optional): [description]. Defaults to 10.
    Returns:
        [type]: [description]
    """
    #MultipartiteRankなどのキーフレーズの手法については以下のサイトが参考になります
    #https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part5.html
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=text, language='ja', normalization=None, stoplist=stopwords)
    extractor.candidate_selection( pos={'NOUN', 'PROPN', 'ADJ', 'NUM'})
    extractor.candidate_weighting(threshold=0.10, method='average', alpha=1.1)
    key_phrase = extractor.get_n_best(n)
    
    return  key_phrase

In [8]:
from spacy.lang.ja import stop_words
spacy_model = spacy.load("ja_ginza")  # GiNZAモデルの読み込み
stopwords = list(stop_words.STOP_WORDS)
print(stopwords[-10:])
nltk.corpus.stopwords.words = stopwords

['なく', 'も', 'ところ', 'もの', 'それ', 'の', 'ご', 'たい', 'おい', 'のみ']


In [9]:
text = ". ".join(r04_ishida)
get_key_phrase(spacy_model, text, n=100, stopwords=stopwords)

[('教材 作成', 0.09755936806166772),
 ('層状 媒質', 0.06405934272182079),
 ('mathematica', 0.060735485513285406),
 ('誘電体 円柱', 0.05455976109737337),
 ('誘電率 推定', 0.054461313725697715),
 ('合成 関数', 0.05308295191254813),
 ('微分積分', 0.05079275303558475),
 ('クラーメル', 0.04943679193116122),
 ('行列式', 0.049062031181767427),
 ('matlab ニューラル ネットワーク', 0.045936827657894086),
 ('探索 アルゴリズム', 0.04592338929339313),
 ('実行 時間', 0.044607783203554234),
 ('線形 代数', 0.04328374574140909),
 ('マイクロ波', 0.04289309401329305),
 ('有理数', 0.04104918966911821),
 ('アルゴリズム', 0.038986663836444904),
 ('パルス', 0.03629098534918045),
 ('九産大 12 号館 建物内', 0.03557986937903461),
 ('プログラミング 環境', 0.03490146023960468),
 ('速度 比較', 0.033247537629381395),
 ('電波 測定', 0.02354965480678551)]

In [11]:
import itertools
from pprint import pprint

for b, y in itertools.product(target_labs[-5:], years[-4:]):
    text = ". ".join(docs[y, b])
    keyphrases = get_key_phrase(spacy_model, text, n=100, stopwords=stopwords)
    print(f"{b}-{y}")
    pprint(keyphrases)


于研-R01
[]
于研-R02
[('音声 会話 システム', 0.20447881468689252),
 ('raspberry pi', 0.20229040900076853),
 ('ブログ 機能', 0.16624832555508953),
 ('kaldi', 0.15772915521631145),
 ('商品 管理 機能', 0.10621740251254869),
 ('日本語 音声 コーパス', 0.08469283458848041),
 ('ギャラリー 機能', 0.07834305843990898)]
于研-R03
[('暗号 資産', 0.10685604154639307),
 ('自動 取引 システム', 0.10573292610490406),
 ('web システム', 0.09012527560509595),
 ('音声 会話 システム', 0.08742302796423763),
 ('はぐ 農園 web システム', 0.08366519180995444),
 ('ギャラリー', 0.07940560712152513),
 ('体型 管理 システム', 0.07580628799401519),
 ('ブログ 機能', 0.07048989348753922),
 ('raspberry pi', 0.0683857080708774),
 ('商品 購入 機能', 0.06792338748114507),
 ('イベント 機能', 0.06141041100984946),
 ('方言 音声 認識 システム', 0.05877749221679121),
 ('ai プログラミング 演習 講義 資料', 0.04399874958767218)]
于研-R04
[('サークル 管理 システム', 0.2569298343176349),
 ('文章 変換 システム', 0.12979646890091476),
 ('サークル 管理 機能', 0.08720844626811852),
 ('イベント 管理 機能', 0.08636659533321363),
 ('ユーザインタフェース', 0.08484863368608771),
 ('メンバ アカウント 管理 機能', 0.082015016

前田研-R03
[('教材 開発', 0.0633124755720323),
 ('睡眠 アンケート', 0.061394279255381705),
 ('pyqtgraph', 0.05832625765696261),
 ('フィルタ 処理', 0.05621286111962764),
 ('心拍 変動 解析', 0.05585611894123983),
 ('寝顔 特徴', 0.053367240066174125),
 ('脳波 データ 取得', 0.05305345833841109),
 ('depth 動画', 0.051616431997937105),
 ('因果性', 0.05037064621319457),
 ('心拍 関連 情報 抽出', 0.04939390054661106),
 ('カルマン フィルタ', 0.049156747239475634),
 ('特徴 抽出', 0.047643232326768356),
 ('非接触 肺年齢 推定', 0.044525797085128375),
 ('リアルタイム eeg マップ 表示 ツール', 0.04434458526393137),
 ('視覚 誘発 電位', 0.043125047370325624),
 ('心電図 データ', 0.03993926150890474),
 ('openbci', 0.039837772564534024),
 ('視覚 刺激 画像', 0.038971050168599544),
 ('呼吸 関連 情報', 0.0340765906751819),
 ('簡易 脳波計', 0.033830720043157146),
 ('文字 入力', 0.031645526046421554)]
前田研-R04
[('特徴 抽出', 0.08518991475176899),
 ('変動 抽出', 0.07758837484790326),
 ('心電図 データ', 0.07658619346533128),
 ('寝顔 形状', 0.07092225609978706),
 ('lstm モデル', 0.07002241473459259),
 ('睡眠 実験', 0.06793452558031178),
 ('独立 成分 分析', 0.0